In [1]:
import os, sys, pickle, random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.cm as cm
from tqdm import tqdm
from pathlib import Path
import mne
if os.getcwd().split("/")[-1] != 'BENDR-XAI': os.chdir("../")
from notebooks.utils import *

In [3]:

(x - minimum) / (maximum - minimum) * 2 - 1

NameError: name 'x' is not defined

In [7]:
result_dict

{'power': {'T0': array([[5.55610354e-23, 7.14817449e-23, 1.57481226e-22, 2.46700281e-23,
          7.03035001e-23, 8.90475342e-23, 3.26582824e-23, 1.87382671e-22,
          3.57166943e-23, 9.91576571e-23, 1.37975138e-22, 5.07553897e-23,
          6.47582975e-23, 1.61529887e-22, 2.48042098e-23, 2.41110558e-23,
          1.16434732e-22, 6.11052421e-23, 3.55679519e-23, 4.30317006e-23,
          2.60227926e-23, 6.04018255e-23, 5.16724993e-23, 7.40021388e-23,
          8.58255635e-23, 4.12248041e-22, 3.62100329e-23, 1.20376208e-22,
          2.10027864e-22, 5.23805331e-23, 4.81892405e-22, 7.73605980e-23,
          2.21842303e-22, 3.48463450e-22, 1.32445229e-22, 9.59438755e-23,
          2.68959206e-22, 2.55350615e-23, 2.49268525e-23, 2.87281488e-22,
          1.35443980e-22, 5.03971810e-23, 6.30640116e-23, 2.36825708e-23,
          1.78699078e-22, 8.67557195e-23],
         [5.85749670e-23, 8.03204109e-23, 1.92004092e-22, 2.47032539e-23,
          8.11397591e-23, 1.05836626e-22, 3.83566912e-

In [11]:
# We use T3 and T4 instead of T7 and T8
channels_order = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']

data = np.load("data/HCPMMP1_combined_8.0_12.0_220503_280423.npy", allow_pickle=True).item()

minimum = -0.00125
maximum = 0.00125

for edf_file, result_dict in tqdm(data.items(), total=len(data)):
    raw = read_TUH_edf(f"/scratch/s194260/tuh_eeg/{edf_file}", low_pass=None)
    try:
        raw = raw.reorder_channels(channels_order)
    except ValueError:
        continue
    
    for anno in result_dict['annotations']['T0']:
        window = get_window(raw, anno).get_data()
        window = (window - minimum) / (maximum - minimum) * 2 - 1

    #for anno in tqdm(result_dict['annotations']['T0']):
    #    window = get_window(raw, anno).get_data()
    #    minimum = np.minimum(window.min(axis=1), minimum)
    #    maximum = np.maximum(window.max(axis=1), maximum)
        
#minimum = minimum.reshape(-1, 1)
#maximum = maximum.reshape(-1, 1)

 17%|█▋        | 34/200 [00:12<01:03,  2.62it/s]


KeyboardInterrupt: 

In [10]:
window.shape

(19, 1024)

In [2]:
# Make a list of path to all edf files in the subfolders of /nobackup/tsal-tmp/tuh_eeg
# files = []

# with tqdm() as pbar:
#     for root, dirs, file in os.walk("/nobackup/tsal-tmp/tuh_eeg"):
#         for name in file:
#             if name.endswith(".edf"):
#                 files.append(os.path.join(root, name))
#                 pbar.update(1)

# Save list of files as pickle
# import pickle
# with open('tuh_files.pkl', 'wb') as f:
#     pickle.dump(files, f)
    
# Load list of files from pickle
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)
    
# random.shuffle(files)

In [3]:
# import pickle
# with open('tuh_files.pkl', 'rb') as f:
#     files = pickle.load(f)

# Copy all files in tuh_final_selected.txt to data/tuh_eeg/
# with open('tuh_final_selected.txt', 'r') as f:
#     selected = f.read().splitlines()

# selected = [file.split("/")[-1] for file in selected]

# Delete all files in /scratch/s194260/tuh_eeg/ that are not in selected
# for file in os.listdir('/scratch/s194260/tuh_eeg/'):
#     if file not in selected:
#         os.remove('/scratch/s194260/tuh_eeg/' + file)